In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
import numpy as np
import os
from IPython.display import display, Javascript
import io
import base64
import cv2
from tensorflow.keras import backend as K
from PIL import Image
from google.colab import output
import google.colab

from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img, save_img
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout,BatchNormalization

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
train_dir = '/content/drive/MyDrive/Signature recognition'


In [ ]:
# Function to count images in each subfolder
def count_images_in_subfolders(directory):
    global imcount
    for user in os.listdir(directory):
        user_dir = os.path.join(directory, user)
        if os.path.isdir(user_dir):
            image_count = int(len([img for img in os.listdir(user_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]))
            print(f"User '{user}' has {image_count} images")
            imcount+=image_count
    print(f"Total images in all users: {imcount}")

# Count and print the number of images in each subfolder
imcount=0

count_images_in_subfolders(train_dir)


User 'bhaski' has 22 images
User 'Barani' has 22 images
User 'Hariharan' has 22 images
Total images in all users: 66


In [ ]:

#. DO  NOT. RUN. THIS. OR. YOU WILL DIE (I better die)

import os
import base64
import io
from PIL import Image
import numpy as np
from IPython.display import display, Javascript
import google.colab.output
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img

train_dir = '/content/drive/MyDrive/Signature recognition'

def preprocess_image(image_data):
    image = Image.open(io.BytesIO(image_data))
    image = image.convert('L')  # Convert to grayscale
    image = image.resize((28, 28))  # Resize to model input size
    img_array = np.array(image) / 255.0  # Normalize to [0, 1]
    img_array = np.expand_dims(img_array, axis=(0, -1))  # Add batch and channel dimensions
    return img_array

def add_subdir(data_url):
    global userName
    global signatureCount

    # Decode base64 image data
    image_data = base64.b64decode(data_url.split(',')[1])
    # Preprocess the image
    img_array = preprocess_image(image_data)

    # Save the image to the user's directory
    user_dir = os.path.join(train_dir, userName)
    if not os.path.exists(user_dir):
        os.makedirs(user_dir)
    img_path = os.path.join(user_dir, f'signature_{signatureCount}.png')
    save_image(img_array, img_path)
    print(f"Signature saved to {img_path}")

    signatureCount += 1  # Increment the count for the next image

    # Perform augmentation after the second signature is saved
    if signatureCount > 2:
        augment_images_for_user(user_dir)

def save_image(img_array, path):
    image = Image.fromarray((img_array.squeeze() * 255).astype(np.uint8))
    image.save(path)

def augment_images_for_user(user_dir):
    # ImageDataGenerator for data augmentation
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        fill_mode='nearest',
        brightness_range=[0.8, 1.2],
        channel_shift_range=0.1
    )

    # Number of augmented images per original image
    num_augmented_images = 10

    # Loop through each image in the user's directory
    for img_name in os.listdir(user_dir):
        img_path = os.path.join(user_dir, img_name)
        if os.path.isfile(img_path) and img_name.endswith(('.png', '.jpg', '.jpeg', '.png')):
            # Load the image
            img = load_img(img_path, color_mode='grayscale')
            x = img_to_array(img)
            x = np.expand_dims(x, axis=0)

            # Create augmented images and save them
            i = 0
            for batch in datagen.flow(x, batch_size=1):
                i += 1
                save_img(os.path.join(user_dir, f'aug_{i}_{img_name}'), batch[0])
                if i >= num_augmented_images:
                    break

DRAW_JS_1 = """
const canvas1 = document.createElement('canvas');
canvas1.width = 400;
canvas1.height = 200;
canvas1.style.border = '1px solid black';
document.body.appendChild(canvas1);

const ctx1 = canvas1.getContext('2d');
ctx1.fillStyle = 'white';
ctx1.fillRect(0, 0, canvas1.width, canvas1.height);

canvas1.onmousedown = function(e) {
  ctx1.beginPath();
  ctx1.moveTo(e.offsetX, e.offsetY);
  canvas1.onmousemove = function(e) {
    ctx1.lineTo(e.offsetX, e.offsetY);
    ctx1.stroke();
  };
};

canvas1.onmouseup = function() {
  canvas1.onmousemove = null;
};

const saveButton1 = document.createElement('button');
saveButton1.innerHTML = 'Save Signature 1';
saveButton1.onclick = function() {
  const dataURL = canvas1.toDataURL('image/png');
  google.colab.kernel.invokeFunction('notebook.add_subdir', [dataURL], {});
};
document.body.appendChild(saveButton1);
"""

DRAW_JS_2 = """
const canvas2 = document.createElement('canvas');
canvas2.width = 400;
canvas2.height = 200;
canvas2.style.border = '1px solid black';
document.body.appendChild(canvas2);

const ctx2 = canvas2.getContext('2d');
ctx2.fillStyle = 'white';
ctx2.fillRect(0, 0, canvas2.width, canvas2.height);

canvas2.onmousedown = function(e) {
  ctx2.beginPath();
  ctx2.moveTo(e.offsetX, e.offsetY);
  canvas2.onmousemove = function(e) {
    ctx2.lineTo(e.offsetX, e.offsetY);
    ctx2.stroke();
  };
};

canvas2.onmouseup = function() {
  canvas2.onmousemove = null;
};

const saveButton2 = document.createElement('button');
saveButton2.innerHTML = 'Save Signature 2';
saveButton2.onclick = function() {
  const dataURL = canvas2.toDataURL('image/png');
  google.colab.kernel.invokeFunction('notebook.add_subdir', [dataURL], {});
};
document.body.appendChild(saveButton2);
"""

def new_user_reg(name):
    global userName
    global signatureCount

    user_dir = os.path.join(train_dir, name)
    if not os.path.exists(user_dir):
        os.makedirs(user_dir)
        print(f"User '{name}' created.")

    userName = name
    signatureCount = 1  # Start with the first signature

    display(Javascript(DRAW_JS_1))
    display(Javascript(DRAW_JS_2))

# Register the add_subdir function for JavaScript to call
google.colab.output.register_callback('notebook.add_subdir', add_subdir)

userreg = input("If you are a new user type 'y': ")
if userreg.lower() == 'y':
    name = input("Enter your name: ")
    new_user_reg(name)


If you are a new user type 'y': y
Enter your name: Hariharan
User 'Hariharan' created.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Signature saved to /content/drive/MyDrive/Signature recognition/Hariharan/signature_1.png
Signature saved to /content/drive/MyDrive/Signature recognition/Hariharan/signature_2.png


In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

train_dir = '/content/drive/MyDrive/Signature recognition'

# Image preprocessing function
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    img = cv2.resize(img, (256, 256))  # Resize to a fixed size
    _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)  # Thresholding
    return img

# Function to load and preprocess images from directory
def load_images_from_directory(directory):
    images = []
    labels = []
    for user in os.listdir(directory):
        user_dir = os.path.join(directory, user)
        if os.path.isdir(user_dir):
            for img_name in os.listdir(user_dir):
                img_path = os.path.join(user_dir, img_name)
                if os.path.isfile(img_path) and img_name.endswith(('.png', '.jpg', '.jpeg')):
                    img = preprocess_image(img_path)
                    if img is not None:
                        images.append(img)
                        labels.append(user)  # Assuming each user folder represents a class

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

# Load and preprocess images
images, labels = load_images_from_directory(train_dir)

# Reshape images to add channel dimension
images = images.reshape(images.shape[0], 256, 256, 1).astype('float32') / 255.0

# Encode labels as integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Convert labels to categorical format
from tensorflow.keras.utils import to_categorical
labels_categorical = to_categorical(labels_encoded)

# Data augmentation generator
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest'
)

# Define model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(len(np.unique(labels)), activation='softmax')  # Output has as many classes as there are users
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping, saving the best model, and reducing learning rate on plateau
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_signature_model.h5', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5, min_lr=0.00001)

# Train the model with data augmentation
model.fit(
    datagen.flow(images, labels_categorical, batch_size=22),
    epochs=len(os.listdir(train_dir) * 10),
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

# Save the trained model
model.save('final_signature_model.h5')


Epoch 1/30
3/3 [==============================] - ETA: 0s - loss: 1.0249 - accuracy: 0.3182

3/3 [==============================] - 13s 3s/step - loss: 1.0249 - accuracy: 0.3182 - lr: 0.0010
Epoch 2/30
3/3 [==============================] - ETA: 0s - loss: 0.8271 - accuracy: 0.6364

3/3 [==============================] - 12s 3s/step - loss: 0.8271 - accuracy: 0.6364 - lr: 0.0010
Epoch 3/30
3/3 [==============================] - ETA: 0s - loss: 0.6828 - accuracy: 0.7879

3/3 [==============================] - 12s 4s/step - loss: 0.6828 - accuracy: 0.7879 - lr: 0.0010
Epoch 4/30
3/3 [==============================] - ETA: 0s - loss: 0.6408 - accuracy: 0.7424

3/3 [==============================] - 11s 3s/step - loss: 0.6408 - accuracy: 0.7424 - lr: 0.0010
Epoch 5/30
3/3 [==============================] - ETA: 0s - loss: 0.3692 - accuracy: 0.8636

...

3/3 [==============================] - 10s 3s/step - loss: 0.3444 - accuracy: 0.8485 - lr: 1.0000e-05
Epoch 28/30
3/3 [==============================] - ETA: 0s - loss: 0.3782 - accuracy: 0.8485

3/3 [==============================] - 11s 3s/step - loss: 0.3782 - accuracy: 0.8485 - lr: 1.0000e-05
Epoch 29/30
3/3 [==============================] - ETA: 0s - loss: 0.3638 - accuracy: 0.8939

3/3 [==============================] - 12s 4s/step - loss: 0.3638 - accuracy: 0.8939 - lr: 1.0000e-05
Epoch 30/30
3/3 [==============================] - ETA: 0s - loss: 0.2447 - accuracy: 0.9242

3/3 [==============================] - 17s 6s/step - loss: 0.2447 - accuracy: 0.9242 - lr: 1.0000e-05


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
import base64
import io
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from google.colab import output
from IPython.display import display, Javascript

# Load your handwriting recognition model
model = load_model('/content/final_signature_model.h5')

# Define user names
user_names = os.listdir(train_dir)
stranger_threshold = 0.75  # Threshold for recognizing a stranger

# JavaScript to create a canvas and capture the image
DRAW_JS = """
const canvas = document.createElement('canvas');
canvas.width = 400;
canvas.height = 200;
canvas.style.border = '1px solid black';
document.body.appendChild(canvas);

const ctx = canvas.getContext('2d');
ctx.fillStyle = 'white';
ctx.fillRect(0, 0, canvas.width, canvas.height);

canvas.onmousedown = function(e) {
  ctx.beginPath();
  ctx.moveTo(e.offsetX, e.offsetY);
  canvas.onmousemove = function(e) {
    ctx.lineTo(e.offsetX, e.offsetY);
    ctx.stroke();
  };
};

canvas.onmouseup = function() {
  canvas.onmousemove = null;
};

const predictButton = document.createElement('button');
predictButton.innerHTML = 'Predict';
predictButton.onclick = function() {
  const dataURL = canvas.toDataURL('image/png');
  google.colab.kernel.invokeFunction('notebook.predict', [dataURL], {});
};
document.body.appendChild(predictButton);

const clearButton = document.createElement('button');
clearButton.innerHTML = 'Clear';
clearButton.onclick = function() {
  ctx.clearRect(0, 0, canvas.width, canvas.height);
};
document.body.appendChild(clearButton);
"""

# Function to preprocess the image
def preprocess_image(image_data):
    image = Image.open(io.BytesIO(image_data))
    image = image.convert('L')  # Convert to grayscale
    image = image.resize((256, 256))  # Resize to model input size
    img_array = np.array(image) / 255.0  # Normalize to [0, 1]
    img_array = np.expand_dims(img_array, axis=(0, -1))  # Add batch and channel dimensions
    return img_array

# Function to predict user and calculate probabilities and RMSE
def predict_user(img_array, user_names, model):
    prediction = model.predict(img_array)
    probabilities = prediction[0]
    rmse = np.sqrt(np.mean(np.square(prediction - np.eye(len(user_names)))))
    max_prob = np.max(probabilities)
    print(f"Probabilities: {probabilities}, Max Probability: {max_prob}, RMSE: {rmse}")
    if max_prob < stranger_threshold:
        return 'Stranger', probabilities, rmse
    predicted_user_index = np.argmax(probabilities)
    return user_names[predicted_user_index], probabilities, rmse

# Function to handle the prediction process
def predict_signature(image_data_url):
    image_data = base64.b64decode(image_data_url.split(',')[1])
    img_array = preprocess_image(image_data)
    print(f"Image array shape: {img_array.shape}")
    predicted_user, probabilities, rmse = predict_user(img_array, user_names, model)
    print(f"Predicted User: {predicted_user}")
    print(f"Probabilities: {probabilities}")
    print(f"RMSE: {rmse}")
    return predicted_user, probabilities, rmse

output.register_callback('notebook.predict', predict_signature)

# Display the drawing canvas and buttons
display(Javascript(DRAW_JS))


<IPython.core.display.Javascript object>

Image array shape: (1, 256, 256, 1)
1/1 [==============================] - 0s 144ms/step
Probabilities: [0.29966864 0.07900655 0.6213248 ], Max Probability: 0.6213247776031494, RMSE: 0.5213511809304071
Predicted User: Stranger
Probabilities: [0.29966864 0.07900655 0.6213248 ]
RMSE: 0.5213511809304071


In [ ]:
import matplotlib.pyplot as plt

# Function to plot images in a grid
def plot_images(images_arr, n_images=4):
    fig, axes = plt.subplots(1, n_images, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img[:,:,0], cmap='gray')
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Get a batch of augmented images
augmented_images, _ = next(train_generator)

# Plot the images
plot_images(augmented_images)


NameError: name 'train_generator' is not defined